# VacationPy

In [1]:
# Data Science
import pandas as pd
import numpy as np
import requests


# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
df = pd.read_csv("output_data/cities.csv")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Sinop Province,41.7711,34.8709,37.09,51,26,9.71,1025,TR,1735876930
1,1,Pego,38.8527,-0.0900,51.94,61,16,10.54,1024,ES,1735876931
2,2,Iqaluit,63.7506,-68.5145,19.13,85,100,0.00,1016,CA,1735876933
3,3,Qaqortoq,60.7167,-46.0333,28.99,94,99,5.73,1029,GL,1735876935
4,4,Utrik,11.2278,169.8474,81.00,78,99,14.61,1009,MH,1735876936


In [3]:
# Date Cleaning w/dates
df["Date"] = pd.to_datetime(df.Date * 1e9)
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Sinop Province,41.7711,34.8709,37.09,51,26,9.71,1025,TR,2025-01-03 04:02:10
1,1,Pego,38.8527,-0.0900,51.94,61,16,10.54,1024,ES,2025-01-03 04:02:11
2,2,Iqaluit,63.7506,-68.5145,19.13,85,100,0.00,1016,CA,2025-01-03 04:02:13
3,3,Qaqortoq,60.7167,-46.0333,28.99,94,99,5.73,1029,GL,2025-01-03 04:02:15
4,4,Utrik,11.2278,169.8474,81.00,78,99,14.61,1009,MH,2025-01-03 04:02:16


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City_ID     57 non-null     int64         
 1   City        57 non-null     object        
 2   Lat         57 non-null     float64       
 3   Lng         57 non-null     float64       
 4   Max Temp    57 non-null     float64       
 5   Humidity    57 non-null     int64         
 6   Cloudiness  57 non-null     int64         
 7   Wind Speed  57 non-null     float64       
 8   Pressure    57 non-null     int64         
 9   Country     56 non-null     object        
 10  Date        57 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(2)
memory usage: 5.0+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"] 
    #Add 'city' to the tooltip 
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 32
max_temp = 80
max_wind = 10

# Drop any rows with null values
df2 = df.dropna()

mask = (df2["Max Temp"] >= min_temp) & (df2["Max Temp"] < max_temp) & (df2["Wind Speed"] < max_wind)
df2 = df2.loc[mask].reset_index(drop=True)

# Display sample data
df2

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Pressure,Country,Date
0,0,Sinop Province,41.7711,34.8709,37.09,51,26,9.71,1025,TR,2025-01-03 04:02:10
1,7,Kapaa,22.0752,-159.3190,73.47,87,29,9.10,1017,US,2025-01-03 04:02:20
2,8,Wailua Homesteads,22.0669,-159.3780,72.12,87,32,8.21,1017,US,2025-01-03 04:02:21
3,9,As Sulayyil,20.4607,45.5779,49.23,39,0,7.67,1028,SA,2025-01-03 04:02:23
4,14,Waitangi,-43.9535,-176.5597,60.82,97,100,5.01,997,NZ,2025-01-03 04:02:28
5,20,Puerto Ayora,-0.7393,-90.3518,74.55,97,28,3.00,1014,EC,2025-01-03 04:02:37
6,21,Greymouth,-42.4667,171.2000,58.66,88,87,6.17,1000,NZ,2025-01-03 04:02:38
7,22,Daloa,6.8774,-6.4502,66.70,42,22,2.51,1011,CI,2025-01-03 04:02:39
8,25,Ushuaia,-54.8000,-68.3000,49.66,71,20,5.75,1007,AR,2025-01-03 04:00:33
9,27,Lamu,-2.2717,40.9020,77.14,82,10,8.28,1013,KE,2025-01-03 04:02:46


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Sinop Province,TR,41.7711,34.8709,51,
1,Pego,ES,38.8527,-0.0900,61,
2,Iqaluit,CA,63.7506,-68.5145,85,
3,Qaqortoq,GL,60.7167,-46.0333,94,
4,Utrik,MH,11.2278,169.8474,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Initialize list for hotel data
hotel_rows = []

# Set API key and base URL for Geoapify
geoapify_key = "57dfac0f50524173899c24f738dcca60"
base_url = "https://api.geoapify.com/v2/places"

# Set radius for search
radius = 10000  # 10km radius

# Print message indicating the start of hotel search
#print("Starting hotel search")

# Iterate through DataFrame rows (df2 assumed to be predefined)
for index, row in df2.iterrows():
    # Extract city, country, latitude, and longitude from the row
    city = row["City"]
    country = row["Country"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Set up parameters for the API request
    params = {
        "categories": "accommodation.hotel",  # Search for hotels
        "limit": 1,                          # Limit to 1 result
        "filter": f"circle:{longitude},{latitude},{radius}",  # Search filter based on circle radius
        "bias": f"proximity:{longitude},{latitude}",  # Proximity bias
        "apiKey": geoapify_key               # API key
    }
    
    try:
        # Make the API request
        response = requests.get(base_url, params=params)
        
        # Check the status code of the response
        if response.status_code == 200:
            data = response.json()
            results = data.get("features", [])
            
            if len(results) > 0:
                # Extract hotel data
                place = results[0]
                address = place.get("properties", {}).get("formatted", None)
                name = place.get("properties", {}).get("name", None)
                distance = place.get("properties", {}).get("distance", None)
                elevation = place.get("properties", {}).get("ele", None)
                website = place.get("properties", {}).get("website", None)
                
                hotel_row = {
                    "city": city,
                    "country": country,
                    "latitude": latitude,
                    "longitude": longitude,
                    "address": address,
                    "name": name,
                    "distance": distance,
                    "elevation": elevation,
                    "website": website
                }
            else:
                # If no results found, store None for relevant fields
                hotel_row = {
                    "city": city,
                    "country": country,
                    "latitude": latitude,
                    "longitude": longitude,
                    "address": None,
                    "name": None,
                    "distance": None,
                    "elevation": None,
                    "website": None
                }
        else:
            # If the API request fails, handle the error
            print(f"Error fetching data for {city}, {country}. Status code: {response.status_code}")
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }

    except requests.exceptions.RequestException as e:
        # Handle any request-related errors
        print(f"Error with request for {city}, {country}: {e}")
        hotel_row = {
            "city": city,
            "country": country,
            "latitude": latitude,
            "longitude": longitude,
            "address": None,
            "name": None,
            "distance": None,
            "elevation": None,
            "website": None
        }

    # Append the result to the hotel_rows list
    hotel_rows.append(hotel_row)

# Convert hotel_rows list into DataFrame (if needed)
hotel_df = pd.DataFrame(hotel_rows)

# Display sample data
hotel_df.head()

,city,country,latitude,longitude,address,name,distance,elevation,website
0,Sinop Province,TR,41.7711,34.8709,None,None,NaN,None,None
1,Kapaa,US,22.0752,-159.3190,"Pono Kai Resort, Pono Kai Street, Kapaa, HI 96...",Pono Kai Resort,157.0,None,http://ponokairesort.com/
2,Wailua Homesteads,US,22.0669,-159.3780,"Hilton Garden Inn Kauai Wailua Bay, Kuhio High...",Hilton Garden Inn Kauai Wailua Bay,4923.0,None,None
3,As Sulayyil,SA,20.4607,45.5779,"10, As Sulayyil, Saudi Arabia",None,1539.0,None,None
4,Waitangi,NZ,-43.9535,-176.5597,"Hotel Chathams, 1 Waitangi Wharf Owenga Road, ...",Hotel Chathams,295.0,None,https://www.hotelchatham.co.nz/


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name"]  # Add 'city' to the tooltip
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name)